Import Libraries & Load Model

In [53]:
import cv2
import numpy as np
import joblib
from skimage.color import rgb2gray
from skimage import io
from skimage.measure import shannon_entropy
import matplotlib.pyplot as plt
# Load the trained Random Forest model
model = joblib.load('model.joblib')
print("Loaded model.joblib")


Loaded model.joblib


Visualize Original vs Sobel‐Processed Image

 Define Feature Extraction:

In [54]:
def extract_sobel_features(image_path):
    """
    Load an image from disk, convert to grayscale,
    apply Sobel edge detection, then compute
    variance, skewness, kurtosis, and entropy.
    Returns a 1×4 feature list.
    """
    # 1. Read and grayscale
    img = io.imread(image_path)
    if img.ndim == 3:
        img = rgb2gray(img)
    # 2. Resize to 400×400 (or your training size)
    img = cv2.resize(img, (400, 400))
    # 3. Sobel in X and Y
    sobel_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    grad = np.hypot(sobel_x, sobel_y)
    # 4. Compute statistical features
    var   = np.var(grad)
    skew  = np.mean(((grad - grad.mean())/grad.std())**3)
    kurt  = np.mean(((grad - grad.mean())/grad.std())**4) - 3
    ent   = shannon_entropy(grad)
    return np.array([[var, skew, kurt, ent]])


Predict Function

In [55]:
def predict_banknote(image_path):
    """
    Given a file path, extract Sobel features,
    apply the loaded model, and return a string.
    """
    features = extract_sobel_features(image_path)
    label = model.predict(features)[0]
    return "FAKE banknote" if label==1 else "REAL banknote"


Run Prediction on Your Image

In [56]:
# Example: prompt for path and predict
img_path = r"D:\Fake money Detective\images.jpg"
result = predict_banknote(img_path)
print("Prediction:", result)

Prediction: FAKE banknote
